# Dakar 2025

*Notes on scraping Dakar 2025 data.*

Example results page at: https://www.dakar.live.worldrallyraidchampionship.com/en/car/2/latest-ranking

Example standings: https://www.dakar.live.worldrallyraidchampionship.com/en/car/2/standings

From inspection of resources loaded via browser developer tools, we can identify the following data feeds:

- groups feed: https://www.dakar.live.worldrallyraidchampionship.com/api/allGroups-2025
- example clazz feed: https://www.dakar.live.worldrallyraidchampionship.com/api/allClazz-2025-A
- example stage feed: https://www.dakar.live.worldrallyraidchampionship.com/api/lastScore-2025-A-1
- example waypoints feed: https://www.dakar.live.worldrallyraidchampionship.com/api/waypoint-2025-A-1
- example news feed: https://www.dakar.live.worldrallyraidchampionship.com/api/publication_en-2025-1
- example withdrawal feed: https://www.dakar.live.worldrallyraidchampionship.com/api/withdrawal-2025-M
 
API filters are: `-{year}-{category}?-{stage}?`


In [1]:
CATEGORIES = {"car":"A", "bike":"M"}

In [2]:
CATEGORY = "car"
STAGE = 2
YEAR = 2025

In [4]:
import pandas as pd
import requests

## Groups

Vehicle groups:

In [227]:
groups_feed_template = "https://www.dakar.live.worldrallyraidchampionship.com/api/allGroups-{year}"

groups_response = requests.get(groups_feed_template.format(**{"year":YEAR}))

In [230]:
_groups_df = pd.json_normalize(groups_response.json())
_groups_df["Year"] = YEAR
_groups_df.head(3)

,categoryGroupLangs,shortLabel,promotionalDisplay,reference,tinyLabel,position,label,liveDisplay,updatedAt,refueling,_bind,_origin,_id,_key,_updatedAt,_parent,color,Year,Stage
0,"[{'text': 'M1000 - Bike', 'locale': 'en', 'var...",cat.name.F_M,False,2025-F-M,M,0,M,False,2025-01-05T20:25:31+01:00,0,allGroups-2025,categoryGroup-2025-F,ae6882fe1fcb5dbef0a7e42d7ca802c1,label,1736183714461,category-2025:0a0fb4dfc03bcb6b0e9c940b5a09ae05,NaN,2025,2
1,"[{'text': 'M1000 - Quad', 'variable': 'cat.nam...",cat.name.F_Q,False,2025-F-Q,Q,1,Q,False,2025-01-05T20:25:31+01:00,0,allGroups-2025,categoryGroup-2025-F,d02ea9bfe0a842c0650bba3c2cf33ecc,label,1736183714461,category-2025:0a0fb4dfc03bcb6b0e9c940b5a09ae05,NaN,2025,2
2,"[{'locale': 'en', 'text': 'M1000 - Car', 'vari...",cat.name.F_A,False,2025-F-A,A,2,A,False,2025-01-05T20:25:31+01:00,0,allGroups-2025,categoryGroup-2025-F,d7cb4906cafa111859658683cf820133,label,1736183714461,category-2025:0a0fb4dfc03bcb6b0e9c940b5a09ae05,NaN,2025,2


In [214]:
_vehicle_labels = (
        groups_df[["categoryGroupLangs"]]
        # categoryGroupLangs is a list of dicts
        # Explode the list over multiple rows, flatten/normalise the dicts,
        # and join the cols back in
        .join(pd.json_normalize(groups_df["categoryGroupLangs"].explode()))
        # Drop the original compound categoryGroupLangs column;
        # the unpacked variable column is a duplicate of the shortLabel
    .drop(["categoryGroupLangs"], axis=1)
    .pivot_table(
    index='variable',
    columns='locale', 
    values='text',
    aggfunc='first'  # Takes first text value if there are duplicates
)
    .reset_index()
    .rename(columns={"variable":"shortLabel"})
    )

_vehicle_labels

locale,shortLabel,ar,en,es,fr
0,cat.name.F_A,M1000 - Car,M1000 - Car,M1000 - Coche,M1000 - Auto
1,cat.name.F_M,M1000 - Bike,M1000 - Bike,M1000 - Moto,M1000 - Moto
2,cat.name.F_Q,M1000 - Quad,M1000 - Quad,M1000 - Quad,M1000 - Quad
3,cat.name.F_T4,NaN,M1000 - SSV,NaN,M1000 - SSV
4,cat.name.F_T5,M1000 - Truck,M1000 - Truck,M1000 - Camión,M1000 - Camion


In [222]:
groups_df = pd.merge(_groups_df, _vehicle_labels, on="shortLabel")
groups_df.drop(["categoryGroupLangs"], axis=1, inplace=True)
groups_df["_label"] = groups_df["en"].str.split("-").str[-1].str.strip()
add_stage_metadata(groups_df)
groups_df

,shortLabel,promotionalDisplay,reference,tinyLabel,position,label,liveDisplay,updatedAt,refueling,_bind,...,_id,_key,_updatedAt,_parent,color,ar,en,es,fr,_label
0,cat.name.F_M,False,2025-F-M,M,0,M,False,2025-01-05T20:25:31+01:00,0,allGroups-2025,...,ae6882fe1fcb5dbef0a7e42d7ca802c1,label,1736183714461,category-2025:0a0fb4dfc03bcb6b0e9c940b5a09ae05,NaN,M1000 - Bike,M1000 - Bike,M1000 - Moto,M1000 - Moto,Bike
1,cat.name.F_Q,False,2025-F-Q,Q,1,Q,False,2025-01-05T20:25:31+01:00,0,allGroups-2025,...,d02ea9bfe0a842c0650bba3c2cf33ecc,label,1736183714461,category-2025:0a0fb4dfc03bcb6b0e9c940b5a09ae05,NaN,M1000 - Quad,M1000 - Quad,M1000 - Quad,M1000 - Quad,Quad
2,cat.name.F_A,False,2025-F-A,A,2,A,False,2025-01-05T20:25:31+01:00,0,allGroups-2025,...,d7cb4906cafa111859658683cf820133,label,1736183714461,category-2025:0a0fb4dfc03bcb6b0e9c940b5a09ae05,NaN,M1000 - Car,M1000 - Car,M1000 - Coche,M1000 - Auto,Car
3,cat.name.F_T5,False,2025-F-T5,C,4,T5,False,2025-01-05T20:25:31+01:00,0,allGroups-2025,...,e0dcecc76fec5cd2cd095a09b821242d,label,1736183714461,category-2025:0a0fb4dfc03bcb6b0e9c940b5a09ae05,NaN,M1000 - Truck,M1000 - Truck,M1000 - Camión,M1000 - Camion,Truck
4,cat.name.F_T4,False,2025-F-T4,SSV,3,T4,False,2025-01-05T20:25:31+01:00,0,allGroups-2025,...,f38a8e2eb62f13bc043106815cb187bd,label,1736183714461,category-2025:0a0fb4dfc03bcb6b0e9c940b5a09ae05,NaN,NaN,M1000 - SSV,NaN,M1000 - SSV,SSV


## Clazz

In [49]:
clazz_template = "https://www.dakar.live.worldrallyraidchampionship.com/api/allClazz-{year}-{category}"

clazz_response = requests.get(clazz_template.format(**{"category":CATEGORIES[CATEGORY], "year":YEAR}))

In [50]:
clazz = pd.json_normalize(clazz_response.json())
clazz["Year"] = YEAR
clazz["Category"] = CATEGORIES[CATEGORY]
clazz

,position,label,refueling,promotionalDisplay,categoryClazzLangs,liveDisplay,shortLabel,updatedAt,reference,_bind,_origin,_id,_key,_updatedAt,_parent,$group,_gets.group,categoryGroupLangs,tinyLabel,color
0,1,U,0,True,"[{'variable': 'cat.name.A_T3_U', 'text': 'T3.U...",False,cat.name.A_T3_U,2025-01-05T20:25:31+01:00,2025-A-T3-U,allClazz-2025-A,categoryClazz-2025-A-T3,18af44f476a4dc9363554ccfe1a9b9fe,_id,1736183714844,categoryGroup-2025-A:15f329900afa29e3e6b099ae6...,categoryGroup-2025-A:15f329900afa29e3e6b099ae6...,$group,NaN,NaN,NaN
1,0,1,0,True,"[{'variable': 'cat.name.A_T3_1', 'text': 'T3.1...",False,cat.name.A_T3_1,2025-01-05T20:25:31+01:00,2025-A-T3-1,allClazz-2025-A,categoryClazz-2025-A-T3,a0a6386a4b9a61b73b036a50966345c0,_id,1736183714844,categoryGroup-2025-A:15f329900afa29e3e6b099ae6...,categoryGroup-2025-A:15f329900afa29e3e6b099ae6...,$group,NaN,NaN,NaN
2,3,T4,0,False,"[{'variable': 'cat.name.A_T4_T4', 'locale': 'e...",False,cat.name.A_T4_T4,2025-01-05T20:25:31+01:00,2025-A-T4-T4,allClazz-2025-A,categoryClazz-2025-A-T4,058d77cc7db191813c30a902a8d5ba7c,_id,1736183714670,categoryGroup-2025-A:423ea731fdcba5cda62c83349...,categoryGroup-2025-A:423ea731fdcba5cda62c83349...,$group,NaN,NaN,NaN
3,0,NO,0,False,"[{'text': 'T4: Modified Production SSV', 'loca...",False,cat.name.A_T4_NO,2025-01-05T20:25:31+01:00,2025-A-T4-NO,allClazz-2025-A,categoryClazz-2025-A-T4,0ec1b5373f8c1fb5ff70ea0590e16c50,_id,1736183714670,categoryGroup-2025-A:423ea731fdcba5cda62c83349...,categoryGroup-2025-A:423ea731fdcba5cda62c83349...,$group,NaN,NaN,NaN
4,2,SSV2,0,False,"[{'variable': 'cat.name.A_T4_SSV2', 'text': 'S...",False,cat.name.A_T4_SSV2,2025-01-05T20:25:31+01:00,2025-A-T4-SSV2,allClazz-2025-A,categoryClazz-2025-A-T4,23ae09bc22535129a9af1e6b3071bc2c,_id,1736183714670,categoryGroup-2025-A:423ea731fdcba5cda62c83349...,categoryGroup-2025-A:423ea731fdcba5cda62c83349...,$group,NaN,NaN,NaN
5,1,SSV1,0,False,"[{'locale': 'en', 'text': 'SSV1', 'variable': ...",False,cat.name.A_T4_SSV1,2025-01-05T20:25:31+01:00,2025-A-T4-SSV1,allClazz-2025-A,categoryClazz-2025-A-T4,9a68ed3c41c5c7a1642df5d93458baa6,_id,1736183714670,categoryGroup-2025-A:423ea731fdcba5cda62c83349...,categoryGroup-2025-A:423ea731fdcba5cda62c83349...,$group,NaN,NaN,NaN
6,0,STK,0,False,"[{'locale': 'en', 'variable': 'cat.name.A_T2_S...",False,cat.name.A_T2_STK,2025-01-05T20:25:31+01:00,2025-A-T2-STK,allClazz-2025-A,categoryClazz-2025-A-T2,0f5f4067ac5f62792280af002c46441c,_id,1736183714664,categoryGroup-2025-A:4dac064bf100bc806b91e7f2e...,categoryGroup-2025-A:4dac064bf100bc806b91e7f2e...,$group,NaN,NaN,NaN
7,1,1,0,False,"[{'locale': 'en', 'variable': 'cat.name.A_T2_1...",False,cat.name.A_T2_1,2025-01-05T20:25:31+01:00,2025-A-T2-1,allClazz-2025-A,categoryClazz-2025-A-T2,ba59bf65075fd967e43a906aa8e26487,_id,1736183714664,categoryGroup-2025-A:4dac064bf100bc806b91e7f2e...,categoryGroup-2025-A:4dac064bf100bc806b91e7f2e...,$group,NaN,NaN,NaN
8,2,2,0,False,"[{'locale': 'en', 'text': 'Stock with expired ...",False,cat.name.A_T2_2,2025-01-05T20:25:31+01:00,2025-A-T2-2,allClazz-2025-A,categoryClazz-2025-A-T2,eb006e11adce44321f6ca8de47012473,_id,1736183714664,categoryGroup-2025-A:4dac064bf100bc806b91e7f2e...,categoryGroup-2025-A:4dac064bf100bc806b91e7f2e...,$group,NaN,NaN,NaN
9,3,U,0,True,"[{'variable': 'cat.name.A_T1_U', 'locale': 'en...",False,cat.name.A_T1_U,2025-01-05T20:25:31+01:00,2025-A-T1-U,allClazz-2025-A,categoryClazz-2025-A-T1,1501ebcbaf3ad27e72aecfba7faa8037,_id,1736183714852,categoryGroup-2025-A:b49155b3f5670d2a907aa01e3...,categoryGroup-2025-A:b49155b3f5670d2a907aa01e3...,$group,NaN,NaN,NaN


In [51]:
clazz["tinyLabel"].unique()

array([nan, 'CHG', 'SSV', 'STK', 'ULT', 'TRK'], dtype=object)

## Waypoints

The waypoints file gives identifiers and kilometer points for each waypoint.

In [52]:
waypoint_url_template = "https://www.dakar.live.worldrallyraidchampionship.com/api/waypoint-{year}-{category}-{stage}"
waypoint_url_template.format(**{"stage":STAGE, "category":CATEGORIES[CATEGORY], "year":YEAR})

'https://www.dakar.live.worldrallyraidchampionship.com/api/waypoint-2025-A-2'

In [53]:
waypoints_response = requests.get(waypoint_url_template.format(**{"stage":1, "category":CATEGORIES[CATEGORY]}))
#waypoints_response.json()

In [54]:
waypoints = pd.json_normalize(waypoints_response.json()[0]["waypoints"])
waypoints["Year"] = YEAR
waypoints["Stage"] = STAGE
waypoints["category"] = CATEGORIES[CATEGORY]
waypoints

,kilometerPoint,checkpoint,code,kilometerPointDisplay,id,hidden,isCHR,isBRP,isASG,isLBL,groups,isFirstDss
0,38.83,1,01207,39.0,125111,False,True,False,False,False,None,NaN
1,91.04,2,01216,92.0,125112,False,True,False,False,False,None,NaN
2,121.18,3,01218,122.0,125113,False,True,False,False,False,None,NaN
3,150.72,4,01220,151.0,125114,False,True,False,False,False,None,NaN
4,181.18,5,01222,182.0,125115,False,True,False,False,False,None,NaN
5,212.54,6,01224,213.0,125116,False,True,False,False,False,None,NaN
6,243.32,7,01227,244.0,125117,False,True,False,False,False,None,NaN
7,287.02,8,01230,288.0,125119,False,True,False,False,False,None,NaN
8,329.07,9,01233,330.0,125120,False,True,False,False,False,None,NaN
9,365.15,10,01237,366.0,125121,True,True,False,False,False,None,NaN


## Timing and results

Timing and results data for each stage.

We can find the category for a vehicle by taking the `team.clazz` value and cross-referencing it as the `_id` in the corresponding `allClazz-2025-{category}` table, where the `label` gives the category label

In [55]:
results_feed_template = "https://www.dakar.live.worldrallyraidchampionship.com/api/lastScore-2025-{category}-{stage}"
results_feed_template.format(**{"stage":STAGE,  "category":CATEGORIES[CATEGORY]})

'https://www.dakar.live.worldrallyraidchampionship.com/api/lastScore-2025-A-2'

In [139]:
results_response = requests.get(results_feed_template.format(**{"stage":1,"category":CATEGORIES[CATEGORY]}))
#results_response.json()

In [140]:
results = pd.json_normalize(results_response.json())
results.rename(columns={"_id":"result_id"}, inplace=True)
add_stage_metadata(results)
results

,fsh,wd,_bind,_updatedAt,result_id,team.bib,team.clazz,team.brand,team.model,team.competitors,...,cs.01ASS.relative,cs.01ASS.bonus,ce.position,ce.absolute,ce.relative,ce.bonus,cg.01ASS.penality,cs.01ASS.penality,ce.penality,team.vehicleImg
0,True,False,lastScore-2025-A-1,1736187300566,lastScore-2025-A-1-427,427,9a68ed3c41c5c7a1642df5d93458baa6,BRP,CAN-AM MAVERICK R,"[{'name': 'B. LEPIETRE', 'firstName': 'BENOIT'...",...,"[6845000, 6845000]",0,"[125, 125]","[23353000, 23353000]","[6845000, 6845000]",0,NaN,NaN,NaN,NaN
1,False,True,lastScore-2025-A-1,1736187298618,lastScore-2025-A-1-634,634,ec2f26ebeb14824160c7204618a5780d,DAF,FAV 85 MX,"[{'name': 'J. ESTEVE ORO', 'firstName': 'JORDI...",...,"[102292000, 102292000]",0,"[196, 197]","[118800000, 118800000]","[102292000, 102292000]",0,79200000.0,79200000.0,79200000.0,NaN
2,True,False,lastScore-2025-A-1,1736187300449,lastScore-2025-A-1-330,330,a0a6386a4b9a61b73b036a50966345c0,TAURUS,T3 MAX,"[{'name': 'A. ALKUWARI', 'firstName': 'AHMED F...",...,"[1672000, 1672000]",0,"[43, 43]","[18180000, 18180000]","[1672000, 1672000]",0,NaN,NaN,NaN,NaN
3,True,False,lastScore-2025-A-1,1736187300453,lastScore-2025-A-1-243,243,f00d7ec8d2d96e9cf11aa515109376cf,MD,OPTIMUS,"[{'name': 'P. THOMASSE', 'firstName': 'PASCAL'...",...,"[2160000, 2160000]",0,"[50, 50]","[18668000, 18668000]","[2160000, 2160000]",0,NaN,NaN,NaN,NaN
4,True,False,lastScore-2025-A-1,1736187300444,lastScore-2025-A-1-404,404,9a68ed3c41c5c7a1642df5d93458baa6,BRP,CAN-AM MAVERICK R,"[{'name': 'F. LOPEZ CONTARDO', 'firstName': 'F...",...,"[1510000, 1510000]",0,"[34, 34]","[18018000, 18018000]","[1510000, 1510000]",0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196,False,False,lastScore-2025-A-1,1736187300382,lastScore-2025-A-1-212,212,96c0869600e0013dbf5f86f60e5c4da4,DACIA,SANDRIDER,"[{'name': 'C. GUTIERREZ', 'firstName': 'CRISTI...",...,"[133000, 133000]",0,"[5, 5]","[16641000, 16641000]","[133000, 133000]",0,NaN,NaN,NaN,NaN
197,True,False,lastScore-2025-A-1,1736187300456,lastScore-2025-A-1-208,208,96c0869600e0013dbf5f86f60e5c4da4,TOYOTA,HILUX,"[{'name': 'B. VANAGAS', 'firstName': 'BENEDIKT...",...,"[1662000, 1662000]",0,"[41, 41]","[18170000, 18170000]","[1662000, 1662000]",0,NaN,NaN,NaN,NaN
198,False,False,lastScore-2025-A-1,1736187300505,lastScore-2025-A-1-313,313,a0a6386a4b9a61b73b036a50966345c0,TAURUS,T3 MAX,"[{'name': 'K. ALJAFLA', 'firstName': 'KHALID',...",...,"[5299000, 5299000]",0,"[105, 105]","[21807000, 21807000]","[5299000, 5299000]",0,NaN,NaN,NaN,NaN
199,False,False,lastScore-2025-A-1,1736187300619,lastScore-2025-A-1-628,628,ec2f26ebeb14824160c7204618a5780d,IVECO,TRAKKER,"[{'name': 'T. GEUENS', 'firstName': 'TOM', 'la...",...,"[15065000, 15065000]",0,"[169, 169]","[31573000, 31573000]","[15065000, 15065000]",0,NaN,NaN,NaN,NaN


In [141]:
# Need a better joining that takes into account year
results = pd.merge(results, clazz[["_id", "label", "shortLabel"]], left_on='team.clazz', right_on='_id')
results

,fsh,wd,_bind,_updatedAt,result_id,team.bib,team.clazz,team.brand,team.model,team.competitors,...,ce.absolute,ce.relative,ce.bonus,cg.01ASS.penality,cs.01ASS.penality,ce.penality,team.vehicleImg,_id,label,shortLabel
0,True,False,lastScore-2025-A-1,1736187300566,lastScore-2025-A-1-427,427,9a68ed3c41c5c7a1642df5d93458baa6,BRP,CAN-AM MAVERICK R,"[{'name': 'B. LEPIETRE', 'firstName': 'BENOIT'...",...,"[23353000, 23353000]","[6845000, 6845000]",0,NaN,NaN,NaN,NaN,9a68ed3c41c5c7a1642df5d93458baa6,SSV1,cat.name.A_T4_SSV1
1,False,True,lastScore-2025-A-1,1736187298618,lastScore-2025-A-1-634,634,ec2f26ebeb14824160c7204618a5780d,DAF,FAV 85 MX,"[{'name': 'J. ESTEVE ORO', 'firstName': 'JORDI...",...,"[118800000, 118800000]","[102292000, 102292000]",0,79200000.0,79200000.0,79200000.0,NaN,ec2f26ebeb14824160c7204618a5780d,2,cat.name.A_T5_2
2,True,False,lastScore-2025-A-1,1736187300449,lastScore-2025-A-1-330,330,a0a6386a4b9a61b73b036a50966345c0,TAURUS,T3 MAX,"[{'name': 'A. ALKUWARI', 'firstName': 'AHMED F...",...,"[18180000, 18180000]","[1672000, 1672000]",0,NaN,NaN,NaN,NaN,a0a6386a4b9a61b73b036a50966345c0,1,cat.name.A_T3_1
3,True,False,lastScore-2025-A-1,1736187300453,lastScore-2025-A-1-243,243,f00d7ec8d2d96e9cf11aa515109376cf,MD,OPTIMUS,"[{'name': 'P. THOMASSE', 'firstName': 'PASCAL'...",...,"[18668000, 18668000]","[2160000, 2160000]",0,NaN,NaN,NaN,NaN,f00d7ec8d2d96e9cf11aa515109376cf,2,cat.name.A_T1_2
4,True,False,lastScore-2025-A-1,1736187300444,lastScore-2025-A-1-404,404,9a68ed3c41c5c7a1642df5d93458baa6,BRP,CAN-AM MAVERICK R,"[{'name': 'F. LOPEZ CONTARDO', 'firstName': 'F...",...,"[18018000, 18018000]","[1510000, 1510000]",0,NaN,NaN,NaN,NaN,9a68ed3c41c5c7a1642df5d93458baa6,SSV1,cat.name.A_T4_SSV1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196,False,False,lastScore-2025-A-1,1736187300382,lastScore-2025-A-1-212,212,96c0869600e0013dbf5f86f60e5c4da4,DACIA,SANDRIDER,"[{'name': 'C. GUTIERREZ', 'firstName': 'CRISTI...",...,"[16641000, 16641000]","[133000, 133000]",0,NaN,NaN,NaN,NaN,96c0869600e0013dbf5f86f60e5c4da4,+,cat.name.A_T1_+
197,True,False,lastScore-2025-A-1,1736187300456,lastScore-2025-A-1-208,208,96c0869600e0013dbf5f86f60e5c4da4,TOYOTA,HILUX,"[{'name': 'B. VANAGAS', 'firstName': 'BENEDIKT...",...,"[18170000, 18170000]","[1662000, 1662000]",0,NaN,NaN,NaN,NaN,96c0869600e0013dbf5f86f60e5c4da4,+,cat.name.A_T1_+
198,False,False,lastScore-2025-A-1,1736187300505,lastScore-2025-A-1-313,313,a0a6386a4b9a61b73b036a50966345c0,TAURUS,T3 MAX,"[{'name': 'K. ALJAFLA', 'firstName': 'KHALID',...",...,"[21807000, 21807000]","[5299000, 5299000]",0,NaN,NaN,NaN,NaN,a0a6386a4b9a61b73b036a50966345c0,1,cat.name.A_T3_1
199,False,False,lastScore-2025-A-1,1736187300619,lastScore-2025-A-1-628,628,ec2f26ebeb14824160c7204618a5780d,IVECO,TRAKKER,"[{'name': 'T. GEUENS', 'firstName': 'TOM', 'la...",...,"[31573000, 31573000]","[15065000, 15065000]",0,NaN,NaN,NaN,NaN,ec2f26ebeb14824160c7204618a5780d,2,cat.name.A_T5_2


In [142]:
results["label"].unique()

array(['SSV1', '2', '1', 'T4', '+', 'U'], dtype=object)

In [143]:
results["team.competitors"].explode()

0      {'name': 'B. LEPIETRE', 'firstName': 'BENOIT',...
0      {'name': 'R. RELMY-MADINSKA', 'firstName': 'RO...
1      {'name': 'J. ESTEVE ORO', 'firstName': 'JORDI'...
1      {'name': 'F. PARDO', 'firstName': 'FRANCESC', ...
1      {'name': 'J. PUJOL FORNOS', 'firstName': 'JORD...
                             ...                        
199    {'name': 'T. GEUENS', 'firstName': 'TOM', 'las...
199    {'name': 'A. ROBINEAU', 'firstName': 'ANTHONY'...
199    {'name': 'S. KOOPMANN', 'firstName': 'SAM', 'l...
200    {'name': 'P. GONÇALVES', 'firstName': 'PEDRO',...
200    {'name': 'H. MAGALHAES', 'firstName': 'HUGO', ...
Name: team.competitors, Length: 446, dtype: object

In [144]:
results.columns

Index(['fsh', 'wd', '_bind', '_updatedAt', 'result_id', 'team.bib',
       'team.clazz', 'team.brand', 'team.model', 'team.competitors',
       'team.vehicle', 'team.w2rc', 'dss.position', 'dss.absolute', 'dss.real',
       'bonif.total', 'bonif.wp', 'cg.01216.position', 'cg.01216.absolute',
       'cg.01216.relative', 'cg.01218.position', 'cg.01218.absolute',
       'cg.01218.relative', 'cg.01224.position', 'cg.01224.absolute',
       'cg.01224.relative', 'cg.01207.position', 'cg.01207.absolute',
       'cg.01207.relative', 'cg.01233.position', 'cg.01233.absolute',
       'cg.01233.relative', 'cg.01230.position', 'cg.01230.absolute',
       'cg.01230.relative', 'cg.01222.position', 'cg.01222.absolute',
       'cg.01222.relative', 'cg.01220.position', 'cg.01220.absolute',
       'cg.01220.relative', 'cg.01ASS.position', 'cg.01ASS.absolute',
       'cg.01ASS.relative', 'cg.01ASS.bonus', 'cg.01ASS.stagePenalty',
       'cg.01227.position', 'cg.01227.absolute', 'cg.01227.relative',
      

In [145]:
results.iloc[0].to_dict()

{'fsh': True,
 'wd': False,
 '_bind': 'lastScore-2025-A-1',
 '_updatedAt': 1736187300566,
 'result_id': 'lastScore-2025-A-1-427',
 'team.bib': 427,
 'team.clazz': '9a68ed3c41c5c7a1642df5d93458baa6',
 'team.brand': 'BRP',
 'team.model': 'CAN-AM MAVERICK R',
 'team.competitors': [{'name': 'B. LEPIETRE',
   'firstName': 'BENOIT',
   'lastName': 'LEPIETRE',
   'role': 'P',
   'gender': 'm',
   'nationality': 'fra',
   'profil': 'https://img.aso.fr/core_app/img-motorSports-dak-png/427-p/186540/0:0,800:800-0-0-70/d0163',
   'profil_sm': 'https://img.aso.fr/core_app/img-motorSports-dak-png/427-p/186540/0:0,800:800-200-0-70/32f43',
   'podium': 'https://img.aso.fr/core_app/img-motorSports-dak-png/427-p/191920/0:0,532:690-0-0-70/8be63',
   'aid': '110add16-df63-4b90-9494-bc3db7d02662'},
  {'name': 'R. RELMY-MADINSKA',
   'firstName': 'RODRIGUE',
   'lastName': 'RELMY-MADINSKA',
   'role': 'C',
   'gender': 'm',
   'nationality': 'fra',
   'podium': 'https://img.aso.fr/core_app/img-motorSports-d

In [146]:
# ce - end of stage status
# cg - overall status at waypoint
# cs - stage status at waypoint
# dss - stage start status

In [155]:
# Claude.ai
import pandas as pd
from typing import Tuple

def normalize_team_competitors(df: pd.DataFrame, year: int=2025) -> Tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame]:
    """
    Transform a DataFrame containing nested competitor lists into two normalized DataFrames
    using vectorized operations.
    
    Args:
        df (pd.DataFrame): Input DataFrame with columns including 'team.bib', 'team.model', 
                          and 'team.competitors' (list of dicts)
    
    Returns:
        Tuple[pd.DataFrame, pd.DataFrame]: (teams_df, competitors_df)
            - teams_df: DataFrame with team information
            - competitors_df: DataFrame with competitor information and team_bib foreign key
            - results_df: DataFrame of results;
    """
    # Create teams DataFrame by dropping the competitors column
    teams_df = df.drop('team.competitors', axis=1)
    
    # Create competitors DataFrame using explode and json_normalize
    competitors_df = (
        df[['team.bib', 'team.competitors']]
        .explode('team.competitors')
        .rename(columns={'team.bib': 'team_bib'})
        .reset_index(drop=True)
    )
    
    # Normalize the dictionary contents and combine with team_bib
    competitors_df = pd.concat([
        competitors_df['team_bib'],
        pd.json_normalize(competitors_df['team.competitors'])
    ], axis=1)

    # Align column names with previous codebase
    competitors_df["Year"] = year
    competitors_df.rename(columns={'team.bib': 'Bib', 'name':'Name'}, inplace=True)

    team_cols =  [c for c in teams_df.columns if c.startswith("team")] + ['label', 'shortLabel']
    teams_df = teams_df[team_cols]
    teams_df.rename(columns={'team.bib': 'Bib'}, inplace=True)

    team_cols.remove("team.bib")
    team_cols.append("team.competitors")
    return teams_df, competitors_df, df.rename(columns={'team.bib': 'Bib'}).drop(team_cols, axis=1)

In [156]:
teams_df, competitors_df, _results = normalize_team_competitors(results)

In [158]:
_results.columns, display(_results.head())

,fsh,wd,_bind,_updatedAt,result_id,Bib,dss.position,dss.absolute,dss.real,bonif.total,...,cs.01ASS.relative,cs.01ASS.bonus,ce.position,ce.absolute,ce.relative,ce.bonus,cg.01ASS.penality,cs.01ASS.penality,ce.penality,_id
0,True,False,lastScore-2025-A-1,1736187300566,lastScore-2025-A-1-427,427,130,40650000,True,0,...,"[6845000, 6845000]",0,"[125, 125]","[23353000, 23353000]","[6845000, 6845000]",0,NaN,NaN,NaN,9a68ed3c41c5c7a1642df5d93458baa6
1,False,True,lastScore-2025-A-1,1736187298618,lastScore-2025-A-1-634,634,171,41910000,True,0,...,"[102292000, 102292000]",0,"[196, 197]","[118800000, 118800000]","[102292000, 102292000]",0,79200000.0,79200000.0,79200000.0,ec2f26ebeb14824160c7204618a5780d
2,True,False,lastScore-2025-A-1,1736187300449,lastScore-2025-A-1-330,330,55,38370000,True,0,...,"[1672000, 1672000]",0,"[43, 43]","[18180000, 18180000]","[1672000, 1672000]",0,NaN,NaN,NaN,a0a6386a4b9a61b73b036a50966345c0
3,True,False,lastScore-2025-A-1,1736187300453,lastScore-2025-A-1-243,243,67,38730000,True,0,...,"[2160000, 2160000]",0,"[50, 50]","[18668000, 18668000]","[2160000, 2160000]",0,NaN,NaN,NaN,f00d7ec8d2d96e9cf11aa515109376cf
4,True,False,lastScore-2025-A-1,1736187300444,lastScore-2025-A-1-404,404,72,38880000,True,0,...,"[1510000, 1510000]",0,"[34, 34]","[18018000, 18018000]","[1510000, 1510000]",0,NaN,NaN,NaN,9a68ed3c41c5c7a1642df5d93458baa6


(Index(['fsh', 'wd', '_bind', '_updatedAt', 'result_id', 'Bib', 'dss.position',
        'dss.absolute', 'dss.real', 'bonif.total', 'bonif.wp',
        'cg.01216.position', 'cg.01216.absolute', 'cg.01216.relative',
        'cg.01218.position', 'cg.01218.absolute', 'cg.01218.relative',
        'cg.01224.position', 'cg.01224.absolute', 'cg.01224.relative',
        'cg.01207.position', 'cg.01207.absolute', 'cg.01207.relative',
        'cg.01233.position', 'cg.01233.absolute', 'cg.01233.relative',
        'cg.01230.position', 'cg.01230.absolute', 'cg.01230.relative',
        'cg.01222.position', 'cg.01222.absolute', 'cg.01222.relative',
        'cg.01220.position', 'cg.01220.absolute', 'cg.01220.relative',
        'cg.01ASS.position', 'cg.01ASS.absolute', 'cg.01ASS.relative',
        'cg.01ASS.bonus', 'cg.01ASS.stagePenalty', 'cg.01227.position',
        'cg.01227.absolute', 'cg.01227.relative', 'cs.01207.position',
        'cs.01207.absolute', 'cs.01207.relative', 'cs.01218.position',
    

In [150]:
teams_df.head()

,Bib,team.clazz,team.brand,team.model,team.vehicle,team.w2rc,team.vehicleImg,label,shortLabel
0,427,9a68ed3c41c5c7a1642df5d93458baa6,BRP,CAN-AM MAVERICK R,BTR,False,NaN,SSV1,cat.name.A_T4_SSV1
1,634,ec2f26ebeb14824160c7204618a5780d,DAF,FAV 85 MX,TIBAU TEAM,False,NaN,2,cat.name.A_T5_2
2,330,a0a6386a4b9a61b73b036a50966345c0,TAURUS,T3 MAX,NASSER RACING,False,NaN,1,cat.name.A_T3_1
3,243,f00d7ec8d2d96e9cf11aa515109376cf,MD,OPTIMUS,MD RALLYE SPORT,False,NaN,2,cat.name.A_T1_2
4,404,9a68ed3c41c5c7a1642df5d93458baa6,BRP,CAN-AM MAVERICK R,CAN-AM FACTORY TEAM,False,NaN,SSV1,cat.name.A_T4_SSV1


In [120]:
competitors_df

,team_bib,Name,firstName,lastName,role,gender,nationality,profil,profil_sm,podium,aid,Year
0,427,B. LEPIETRE,BENOIT,LEPIETRE,P,m,fra,https://img.aso.fr/core_app/img-motorSports-da...,https://img.aso.fr/core_app/img-motorSports-da...,https://img.aso.fr/core_app/img-motorSports-da...,110add16-df63-4b90-9494-bc3db7d02662,2025
1,427,R. RELMY-MADINSKA,RODRIGUE,RELMY-MADINSKA,C,m,fra,NaN,NaN,https://img.aso.fr/core_app/img-motorSports-da...,63cb271c-1c74-4f07-82b4-c3df3c425aa7,2025
2,634,J. ESTEVE ORO,JORDI,ESTEVE ORO,P,m,esp,https://img.aso.fr/core_app/img-motorSports-da...,https://img.aso.fr/core_app/img-motorSports-da...,https://img.aso.fr/core_app/img-motorSports-da...,36106f23-d545-47b9-8565-983cce7550d3,2025
3,634,F. PARDO,FRANCESC,PARDO,C,m,esp,https://img.aso.fr/core_app/img-motorSports-da...,https://img.aso.fr/core_app/img-motorSports-da...,https://img.aso.fr/core_app/img-motorSports-da...,43c1ce16-ccee-4ac2-ae16-b89e82a43183,2025
4,634,J. PUJOL FORNOS,JORDI,PUJOL FORNOS,M,m,esp,https://img.aso.fr/core_app/img-motorSports-da...,https://img.aso.fr/core_app/img-motorSports-da...,https://img.aso.fr/core_app/img-motorSports-da...,7700b503-68b6-4aca-a63a-96047071c2e9,2025
...,...,...,...,...,...,...,...,...,...,...,...,...
441,628,T. GEUENS,TOM,GEUENS,P,m,bel,https://img.aso.fr/core_app/img-motorSports-da...,https://img.aso.fr/core_app/img-motorSports-da...,https://img.aso.fr/core_app/img-motorSports-da...,30d267bb-e624-4ac2-9886-e6e63709c0e3,2025
442,628,A. ROBINEAU,ANTHONY,ROBINEAU,C,m,fra,https://img.aso.fr/core_app/img-motorSports-da...,https://img.aso.fr/core_app/img-motorSports-da...,https://img.aso.fr/core_app/img-motorSports-da...,bd2e9505-c05c-408e-8f76-620b2e61607a,2025
443,628,S. KOOPMANN,SAM,KOOPMANN,M,m,fra,https://img.aso.fr/core_app/img-motorSports-da...,https://img.aso.fr/core_app/img-motorSports-da...,https://img.aso.fr/core_app/img-motorSports-da...,80df715b-dad9-44f8-b4a4-90631d782835,2025
444,318,P. GONÇALVES,PEDRO,GONÇALVES,P,m,prt,https://img.aso.fr/core_app/img-motorSports-da...,https://img.aso.fr/core_app/img-motorSports-da...,https://img.aso.fr/core_app/img-motorSports-da...,4cd2f99a-77e1-46a8-853b-15ac02f2d196,2025


In [235]:
id_column="_id"
point_cols = [col for col in _results.columns if col.startswith(('cg', 'cs'))]
# Melt only the point-specific columns
melted = _results[[id_column, "Bib", *point_cols]].melt(id_vars=[id_column, "Bib"]).dropna()
melted = melted[melted['variable'].str.contains('position|absolute|relative')]
melted["type"] = melted["variable"].str.split('.').str[0]
melted["waypoint"] = melted["variable"].str.extract(r'\.([^\.]+)\.')
melted["metric"] = melted["variable"].str.split('.').str[-1]

melted[['value_0', 'value_1']] = pd.DataFrame(
        melted['value'].tolist(), 
        index=melted.index
    )
# TO DO - drop: variable, value, _id
# TO DO - have a backup table of _id and Bib
# TO DO - add: Year, Stage
melted

,_id,Bib,variable,value,type,waypoint,metric,value_0,value_1
0,9a68ed3c41c5c7a1642df5d93458baa6,427,cg.01216.position,"[128, 128]",cg,01216,position,128,128
1,ec2f26ebeb14824160c7204618a5780d,634,cg.01216.position,"[194, 194]",cg,01216,position,194,194
2,a0a6386a4b9a61b73b036a50966345c0,330,cg.01216.position,"[53, 53]",cg,01216,position,53,53
3,f00d7ec8d2d96e9cf11aa515109376cf,243,cg.01216.position,"[60, 60]",cg,01216,position,60,60
4,9a68ed3c41c5c7a1642df5d93458baa6,404,cg.01216.position,"[65, 65]",cg,01216,position,65,65
...,...,...,...,...,...,...,...,...,...
12457,96c0869600e0013dbf5f86f60e5c4da4,212,cs.01ASS.relative,"[133000, 133000]",cs,01ASS,relative,133000,133000
12458,96c0869600e0013dbf5f86f60e5c4da4,208,cs.01ASS.relative,"[1662000, 1662000]",cs,01ASS,relative,1662000,1662000
12459,a0a6386a4b9a61b73b036a50966345c0,313,cs.01ASS.relative,"[5299000, 5299000]",cs,01ASS,relative,5299000,5299000
12460,ec2f26ebeb14824160c7204618a5780d,628,cs.01ASS.relative,"[15065000, 15065000]",cs,01ASS,relative,15065000,15065000


In [234]:
melted["type"].unique()

array(['cg', 'cs', 'ce'], dtype=object)

In [162]:
xx = (melted[["_id", "Bib",  "waypoint", "metric",	"value_0","value_1"]].pivot(    index=[id_column, "Bib",'waypoint'],   columns='metric',     values=['value_0', 'value_1'] ) )
xx

value_0                     \
metric                                         absolute position  relative   
_id                              Bib waypoint                                
058d77cc7db191813c30a902a8d5ba7c 400 01207      1532000       46    186000   
                                     01216      3760000       49    433000   
                                     01218      5124000       46    554000   
                                     01220      6461000       44    571000   
                                     01222      7846000       36    659000   
...                                                 ...      ...       ...   
fa2b8d6b8d84480b3fd11a2fd829b2d6 633 01224     14800000      169   6208000   
                                     01227     17597000      165   7354000   
                                     01230     19989000      163   8451000   
                                     01233     23121000      171   9893000   
                                     01ASS     28683000      158  12175000   

                                                value_1                     
metric                                         absolute position  relative  
_id                              Bib waypoint                               
058d77cc7db191813c30a902a8d5ba7c 400 01207      1532000       46    186000  
                                     01216      3760000       49    433000  
                                     01218      5124000       46    554000  
                                     01220      6461000       44    571000  
                                     01222      7846000       36    659000  
...                                                 ...      ...       ...  
fa2b8d6b8d84480b3fd11a2fd829b2d6 633 01224     14800000      169   6208000  
                                     01227     17597000      165   7354000  
                                     01230     19989000      163   8451000  
                                     01233     23121000      171   9893000  
                                     01ASS     28683000      158  12175000  

[1951 rows x 6 columns]

In [27]:
xx["value_0"]
   

metric                          absolute  position  relative
_id                   waypoint                              
lastScore-2025-M-1-1  01207      2485000         3     16000
                      01216      4564000         3     33000
                      01218      5911000         1         0
                      01220      7272000         3     60000
                      01222      8590000         2     14000
...                                  ...       ...       ...
lastScore-2025-M-1-99 01224     12284000        46   2268000
                      01227     14358000        44   2696000
                      01230     15892000        46   2955000
                      01233     18013000        45   3268000
                      01ASS     22219000        49   4321000

[1304 rows x 3 columns]

## Original Database


In [28]:
import sqlite3
from sqlite_utils import Database

dbname = 'dakar_2020.db'

conn = sqlite3.connect(dbname)
db = Database(conn)

In [29]:
db.tables

[<Table teams (Year, Bib, Team)>,
 <Table crew (Year, Bib, Num, Name, Country)>,
 <Table vehicles (Year, Bib, VehicleType, Brand)>,
 <Table stagestats (Year, Stage, Start, Liaison, Special, AtStart, Left, Arrived, LatestWP, LeaderLatestWP, NumLatestWP, BibLatestWP, NameLatestWP, Vehicle, StageDist)>,
 <Table ranking (Year, Stage, Type, Pos, Bib, VehicleType, Crew, Brand, Time_raw, TimeInS, Gap_raw, GapInS, Penalty_raw, PenaltyInS)>,
 <Table waypoints (Year, Stage, Bib, Pos, Waypoint, WaypointOrder, WaypointRank, WaypointPos, Time_raw, TimeInS, Gap_raw, GapInS, WaypointDist, VehicleType, splitS)>]

In [30]:
 pd.DataFrame(db["teams"].rows).head()

,Year,Bib,Team
0,2020,1,RED BULL KTM FACTORY TEAM
1,2020,2,RED BULL KTM FACTORY TEAM
2,2020,3,RED BULL KTM FACTORY TEAM
3,2020,4,MONSTER ENERGY YAMAHA RALLY TEAM
4,2020,5,ROCKSTAR ENERGY HUSQVARNA FACTORY RACING


In [31]:
 pd.DataFrame(db["crew"].rows).head()

,Year,Bib,Num,Name,Country
0,2020,1,0,T. PRICE,Australia
1,2020,2,0,M. WALKNER,Austria
2,2020,3,0,S. SUNDERLAND,United Kingdom
3,2020,4,0,A. VAN BEVEREN,France
4,2020,5,0,P. QUINTANILLA,Chile


In [32]:
 pd.DataFrame(db["stagestats"].rows).head()

,Year,Stage,Start,Liaison,Special,AtStart,Left,Arrived,LatestWP,LeaderLatestWP,NumLatestWP,BibLatestWP,NameLatestWP,Vehicle,StageDist
0,2020,1,07:20,433km,319km,144,144,140,ass,001 PRICE,140,1,PRICE,Moto,319
1,2020,1,08:34,433km,319km,23,23,22,ass,250 CASALE,22,250,CASALE,Quad,319
2,2020,1,09:20,433km,319km,83,82,81,ass,319 ZALA,81,319,ZALA,Car,319
3,2020,1,10:49,433km,319km,46,46,45,ass,419 DOMZALA,45,419,DOMZALA,SSV,319
4,2020,1,11:50,433km,319km,46,41,40,ass,516 SHIBALOV,40,516,SHIBALOV,Truck,319


In [33]:
 pd.DataFrame(db["ranking"].rows).head()

,Year,Stage,Type,Pos,Bib,VehicleType,Crew,Brand,Time_raw,TimeInS,Gap_raw,GapInS,Penalty_raw,PenaltyInS
0,2020,1,stage,1,319,car,V. ZALA S. JURGELENAS AGRORODEO,MINI,03:19:04,11944.0,0:00:00,0.0,00:00:00,0.0
1,2020,1,stage,2,302,car,S. PETERHANSEL P. FIUZA BAHRAIN JCW X-RAID TEAM,MINI,03:21:18,12078.0,0:02:14,134.0,00:00:00,0.0
2,2020,1,stage,3,305,car,C. SAINZ L. CRUZ BAHRAIN JCW X-RAID TEAM,MINI,03:21:54,12114.0,0:02:50,170.0,00:00:00,0.0
3,2020,1,stage,4,300,car,N. AL-ATTIYAH M. BAUMEL TOYOTA GAZOO RACING,TOYOTA,03:24:37,12277.0,0:05:33,333.0,00:00:00,0.0
4,2020,1,stage,5,307,car,B. TEN BRINKE T. COLSOUL TOYOTA GAZOO RACING,TOYOTA,03:25:34,12334.0,0:06:30,390.0,00:00:00,0.0


In [34]:
 pd.DataFrame(db["waypoints"].rows).head()

,Year,Stage,Bib,Pos,Waypoint,WaypointOrder,WaypointRank,WaypointPos,Time_raw,TimeInS,Gap_raw,GapInS,WaypointDist,VehicleType,splitS
0,2020,1,319,1.0,01_km47,1,11.0,10,00:36:18,2178.0,00:02:18,138.0,47,car,2178.0
1,2020,1,302,2.0,01_km47,1,7.0,7,00:35:30,2130.0,00:01:30,90.0,47,car,2130.0
2,2020,1,305,3.0,01_km47,1,4.0,4,00:34:43,2083.0,00:00:43,43.0,47,car,2083.0
3,2020,1,300,4.0,01_km47,1,1.0,1,00:34:00,2040.0,00:00:00,0.0,47,car,2040.0
4,2020,1,307,5.0,01_km47,1,8.0,8,00:36:01,2161.0,00:02:01,121.0,47,car,2161.0


In [35]:
from Dakar_Rally_2021 import dbfy

conn25 = sqlite3.connect("test2025.db")

c25 = conn25.cursor()

setup25_sql= 'dakar25.sql'

with open(setup25_sql,'r') as f:
     txt = f.read()
     c25.executescript(txt)

db25 =  Database(conn)

OperationalError: expressions prohibited in PRIMARY KEY and UNIQUE constraints

In [ ]:
competitors_df.columns

In [ ]:
competitors_df.head()